<a href="https://colab.research.google.com/github/kkusert/work17-10-68/blob/main/%E0%B8%87%E0%B8%B2%E0%B8%99%E0%B8%AA%E0%B8%96%E0%B8%B4%E0%B8%95%E0%B8%B4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
import math
from collections import Counter

def mean(data):
    return sum(data)/len(data) if len(data) > 0 else 0

def median(data):
    if not data: return 0
    sorted_data = sorted(data)
    n = len(sorted_data)
    if n % 2 == 0:
        return (sorted_data[n//2 - 1] + sorted_data[n//2]) / 2
    else:
        return sorted_data[n//2]

def mode(data):
    if not data: return None
    freq = Counter(data)
    max_freq = max(freq.values())
    modes = [k for k, v in freq.items() if v == max_freq]
    if len(modes) == len(data):
        return None  # ไม่มีฐานนิยม
    return modes

def mean_deviation(data):
    if len(data) == 0: return 0
    x_bar = mean(data)
    return sum(abs(x - x_bar) for x in data) / len(data)

def variance(data):
    if len(data) <= 1: return 0
    x_bar = mean(data)
    return sum((x - x_bar)**2 for x in data) / (len(data)-1)

def std_dev(data):
    return math.sqrt(variance(data))

# ---------------------------
# ส่วนที่ 2: แจกแจงความถี่ (เพิ่มสัดส่วนและร้อยละ)
# ---------------------------

def grouped_statistics(data, num_classes):
    if len(data) == 0 or num_classes <= 0:
        return {}

    data.sort()
    min_val, max_val = min(data), max(data)
    R = max_val - min_val
    i = math.ceil(R / num_classes) if num_classes != 0 else 1

    # สร้างช่วงชั้น
    classes = []
    lower = min_val
    for _ in range(num_classes):
        upper = lower + i - 1
        classes.append((lower, upper))
        lower = upper + 1

    # ความถี่ (f)
    freq = [sum(1 for x in data if lo <= x <= hi) for lo, hi in classes]
    total_f = sum(freq)

    # ความถี่สะสม (F)
    cum_freq = []
    running = 0
    for f in freq:
        running += f
        cum_freq.append(running)

    # จุดกึ่งกลาง (x)
    mid = [(lo + hi)/2 for lo, hi in classes]

    # fx, สัดส่วน, ร้อยละ
    fx = [mid[i] * freq[i] for i in range(num_classes)]
    prop = [f / total_f for f in freq]
    perc = [p * 100 for p in prop]

    # ค่าพื้นฐาน
    mean_g = sum(fx) / total_f
    fx2 = [freq[i] * (mid[i]**2) for i in range(num_classes)]
    variance_g = (sum(fx2) / total_f) - (mean_g**2)
    sd_g = math.sqrt(variance_g)

    # มัธยฐาน (Md)
    N = total_f
    median_pos = N / 2
    Md = None
    for j, F in enumerate(cum_freq):
        if F >= median_pos:
            Lm = classes[j][0] - 0.5
            F_prev = cum_freq[j - 1] if j > 0 else 0
            fm = freq[j]
            Md = Lm + ((median_pos - F_prev) / fm) * i if fm != 0 else None
            break

    # ฐานนิยม (Mo)
    max_f = max(freq)
    modal_index = freq.index(max_f)
    L = classes[modal_index][0] - 0.5
    d1 = freq[modal_index] - (freq[modal_index-1] if modal_index > 0 else 0)
    d2 = freq[modal_index] - (freq[modal_index+1] if modal_index < num_classes-1 else 0)
    Mo = L + (d1 / (d1 + d2)) * i if (d1 + d2) != 0 else None

    # -----------------------------
    # คำนวณควอไทล์ Q1, Q3 และ Q.D.
    # -----------------------------
    def find_quartile(pos):
        for j, F in enumerate(cum_freq):
            if F >= pos:
                Lq = classes[j][0] - 0.5
                F_prev = cum_freq[j-1] if j > 0 else 0
                fq = freq[j]
                return Lq + ((pos - F_prev) / fq) * i
        return None

    Q1 = find_quartile(N / 4)
    Q3 = find_quartile(3 * N / 4)
    QD = (Q3 - Q1) / 2 if Q1 and Q3 else None

    return {
        "classes": classes,
        "freq": freq,
        "cum_freq": cum_freq,
        "mid": mid,
        "fx": fx,
        "prop": prop,
        "perc": perc,
        "mean": mean_g,
        "variance": variance_g,
        "sd": sd_g,
        "Mo": Mo,
        "Md": Md,
        "Q1": Q1,
        "Q3": Q3,
        "QD": QD,
        "total_f": total_f
    }

# ---------------------------
# เมนูหลัก
# ---------------------------

def main():
    print("=== โปรแกรมคำนวณค่าสถิติ ===")
    print("1. ข้อมูลดิบ (ไม่แจกแจงความถี่)")
    print("2. ข้อมูลแจกแจงความถี่")
    choice = input("เลือกโหมด (1 หรือ 2): ")

    if choice == "1":
        data = []
        print("ป้อนข้อมูล (1-999) สูงสุด 100 ค่า พิมพ์ 0 เพื่อหยุด:")
        while True:
            try:
                x = int(input("> "))
                if x == 0: break
                if 1 <= x <= 999:
                    data.append(x)
            except:
                print("กรุณาป้อนตัวเลขเท่านั้น")

        if len(data) == 0:
            print("❌ ไม่มีข้อมูล")
            return

        print("\n--- ผลลัพธ์ข้อมูลดิบ ---")
        print("จำนวนข้อมูล =", len(data))
        print("ค่าสูงสุด(Max) =", max(data))
        print("ค่าต่ำสุด(Min) =", min(data))
        print("ค่ามัชฌิมเลขคณิต(Mean) =", round(mean(data), 3))
        print("ค่ามัธยฐาน(Median) =", round(median(data), 3))
        print("ค่าฐานนิยม(Mode) =", mode(data))
        print("ค่าความเบี่ยงเบนเฉลี่ย(Mean Deviation) =", round(mean_deviation(data), 3))
        print("ค่าความแปรปรวน(Variance) =", round(variance(data), 3))
        print("ค่าความเบี่ยงเบนมาตรฐาน(Std Deviation) =", round(std_dev(data), 3))

    elif choice == "2":
        data = []
        print("ป้อนข้อมูล (1-999) พิมพ์ 0 เพื่อหยุด:")
        while True:
            try:
                x = int(input("> "))
                if x == 0: break
                if 1 <= x <= 999:
                    data.append(x)
            except:
                print("กรุณาป้อนตัวเลขเท่านั้น")

        if len(data) == 0:
            print("❌ ไม่มีข้อมูล")
            return

        num_classes = int(input("จำนวนชั้น (3-15): "))
        result = grouped_statistics(data, num_classes)
        if not result:
            print("❌ ข้อมูลไม่เพียงพอหรือจำนวนชั้นไม่ถูกต้อง")
            return

        print("\n=== ตารางแจกแจงความถี่ ===")
        print("ช่วงคะแนน | ขีดจำกัดล่าง(Lo) | ขีดจำกัดบน(Ho) | จุดกลางชั้น(x) | ความถี่(f) | ความถี่สะสม(F) | fx | สัดส่วน | ร้อยละ")
        print("-" * 80)
        for i in range(len(result["classes"])):
            lo, hi = result["classes"][i]
            print(f"{lo:>3}-{hi:<3} | {lo:>3} | {hi:>3} | {result['mid'][i]:>5.1f} | "
                  f"{result['freq'][i]:>3} | {result['cum_freq'][i]:>3} | {result['fx'][i]:>7.2f} | "
                  f"{result['prop'][i]:>7.3f} | {result['perc'][i]:>6.2f}")

        print("-" * 80)
        print(f"รวมทั้งหมด: {result['total_f']} | fx รวม = {sum(result['fx']):.2f}")
        print("\n--- ค่าสถิติ ---")
        print(f"ค่าความเบี่ยงเบนควอไทล์(Quartile Deviation: Q.D.) = {result['QD']:.3f}")
        print(f"ค่าเฉลี่ย(Mean) = {result['mean']:.3f}")
        print(f"ค่าความแปรปรวน(Variance) = {result['variance']:.3f}")
        print(f"ค่าความเบี่ยงเบนมาตรฐาน(Std Deviation) = {result['sd']:.3f}")
        print(f"ค่ามัธยฐาน(Median) = {result['Md']:.3f}")
        print(f"ค่าฐานนิยม(Mode) = {result['Mo']:.3f}" if result["Mo"] else "Mode = ไม่มี")

    else:
        print("เลือกไม่ถูกต้อง")

if __name__ == "__main__":
    main()

=== โปรแกรมคำนวณค่าสถิติ ===
1. ข้อมูลดิบ (ไม่แจกแจงความถี่)
2. ข้อมูลแจกแจงความถี่
เลือกโหมด (1 หรือ 2): 1
ป้อนข้อมูล (1-999) สูงสุด 100 ค่า พิมพ์ 0 เพื่อหยุด:
> 300
> 200
> 100
> 1
> 0

--- ผลลัพธ์ข้อมูลดิบ ---
จำนวนข้อมูล = 4
ค่าสูงสุด(Max) = 300
ค่าต่ำสุด(Min) = 1
ค่ามัชฌิมเลขคณิต(Mean) = 150.25
ค่ามัธยฐาน(Median) = 150.0
ค่าฐานนิยม(Mode) = None
ค่าความเบี่ยงเบนเฉลี่ย(Mean Deviation) = 99.75
ค่าความแปรปรวน(Variance) = 16566.917
ค่าความเบี่ยงเบนมาตรฐาน(Std Deviation) = 128.713
